In [1]:
!pip install gensim

In [2]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving mentalhealth.csv to mentalhealth.csv


In [4]:
df=pd.read_csv('mentalhealth.csv')

In [5]:
df.head()

,Question_ID,Questions,Answers
0,NaN,hi,"Hi, How can I help you ?"
1,NaN,hello,"Hello, How can I help you ?"
2,NaN,good morning,"Good morning, how can I help you ?"
3,NaN,good afternoon,"Good night, how can I help you ?"
4,NaN,good evening,"Good evening, how can I help you ?"


In [6]:
df=df[['Questions','Answers']]

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [74]:
def text_normalization(text):
  text = str(text).lower()
  l=[]
  doc=nlp(text)
  for i in doc:
     if not i.is_punct:
      l.append(i.text)
  return l

In [76]:
df['normalized_text']=df['Questions'].apply(text_normalization)

In [75]:
# print(text_normalization('Good evening, how can I help you ?'))

['good', 'evening', 'how', 'can', 'i', 'help', 'you']


In [77]:
df

,Questions,Answers,normalized_text
0,hi,"Hi, How can I help you ?",[hi]
1,hello,"Hello, How can I help you ?",[hello]
2,good morning,"Good morning, how can I help you ?","[good, morning]"
3,good afternoon,"Good night, how can I help you ?","[good, afternoon]"
4,good evening,"Good evening, how can I help you ?","[good, evening]"
...,...,...,...
158,How can I cope with the emotional pain of a br...,It's important to allow yourself to feel and p...,"[how, can, i, cope, with, the, emotional, pain..."
159,How do I handle constant thoughts about my ex-...,Practice mindfulness to redirect your thoughts...,"[how, do, i, handle, constant, thoughts, about..."
160,What are healthy ways to move on after a breakup?,"Focus on self-discovery, set new goals, and es...","[what, are, healthy, ways, to, move, on, after..."
161,Is it normal to feel guilt after a breakup?,"Yes, it's common. Reflect on your actions, lea...","[is, it, normal, to, feel, guilt, after, a, br..."


In [21]:
from gensim.models.doc2vec import Doc2Vec , TaggedDocument


In [78]:
tagged_data=[TaggedDocument(d,[i]) for i,d in enumerate(df['normalized_text'])]

In [79]:
tagged_data

[TaggedDocument(words=['hi'], tags=[0]),
 TaggedDocument(words=['hello'], tags=[1]),
 TaggedDocument(words=['good', 'morning'], tags=[2]),
 TaggedDocument(words=['good', 'afternoon'], tags=[3]),
 TaggedDocument(words=['good', 'evening'], tags=[4]),
 TaggedDocument(words=['good', 'night'], tags=[5]),
 TaggedDocument(words=['what', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness'], tags=[6]),
 TaggedDocument(words=['who', 'does', 'mental', 'illness', 'affect'], tags=[7]),
 TaggedDocument(words=['what', 'are', 'some', 'of', 'the', 'warning', 'signs', 'of', 'mental', 'illness'], tags=[8]),
 TaggedDocument(words=['can', 'people', 'with', 'mental', 'illness', 'recover'], tags=[9]),
 TaggedDocument(words=['what', 'should', 'i', 'do', 'if', 'i', 'know', 'someone', 'who', 'appears', 'to', 'have', 'the', 'symptoms', 'of', 'a', 'mental', 'disorder'], tags=[10]),
 TaggedDocument(words=['how', 'can', 'i', 'find', 'a', 'mental', 'health', 'professional', 'for', 'myself', 'or', 'my', 'chi

In [134]:
model=Doc2Vec(tagged_data,vector_size=20,min_count=1,workers=4,window=2,epochs=200)

In [128]:
model

In [129]:
test_doc=text_normalization('Good Morning!')

In [83]:
test_doc

['good', 'morning']

In [135]:
test_doc_vector=model.infer_vector(test_doc)

In [136]:
pred=model.docvecs.most_similar(positive=[test_doc_vector])

<ipython-input-136-380263734f74>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  pred=model.docvecs.most_similar(positive=[test_doc_vector])


<ipython-input-56-380263734f74>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  pred=model.docvecs.most_similar(positive=[test_doc_vector])


In [87]:
df['Answers'][pred[0][0]]

'good morning'

In [137]:
def Test(que):
  test_doc=text_normalization(que.lower())
  test_doc_vector=model.infer_vector(test_doc)
  pred=model.docvecs.most_similar(positive=[test_doc_vector])
  # print(pred)
  return df['Answers'][pred[0][0]]

In [ ]:
import telegram.ext
def start(update,context):
  update.message.reply_text("Hello!")
def mess_reply(update,context):
  update.message.reply_text(Test(update.message.text))
token="6819614990:AAH7Sbn0yD7H5KuoNDu1UTx0Qa1PN1egGJc"
updater=telegram.ext.Updater(token,use_context=True)
dispatcher=updater.dispatcher
dispatcher.add_handler(telegram.ext.CommandHandler('start',start))
dispatcher.add_handler(telegram.ext.MessageHandler(telegram.ext.Filters.text,mess_reply))
updater.start_polling()
updater.idle()

<ipython-input-137-612404431742>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  pred=model.docvecs.most_similar(positive=[test_doc_vector])


In [118]:
print(Test('good morning'))

g


<ipython-input-115-131ef5055e32>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  pred=model.docvecs.most_similar(positive=[test_doc_vector])


In [120]:
!pip install python-telegram-bot==13.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.2
    Uninstalling cachetools-5.3.2:
      Successfully uninstalled cachetools-5.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
